In [7]:
import pandas as pd
import os
import re
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [8]:
def parse_logs(filename):
    with open(filename) as f:
        content = f.readlines()        
        
    content = [x.strip() for x in content] 
    results = []
    
    for line in content:    
        try:
            data = {}
            data['passed'] = bool(re.search("passed", line, flags=re.IGNORECASE))
            data['name'] = re.search("isis3_[A-z]+|[A-z]+_app_[A-z]+", line, flags=re.IGNORECASE).group(0)
            data['test_id'] = int(re.search("#[0-9]+", line, flags=re.IGNORECASE).group(0)[1:])
            data['time'] = float(re.search("[0-9]+\.[0-9]+", line, flags=re.IGNORECASE).group(0))
            results.append(data)
        except Exception as e:
            pass
    
    return pd.DataFrame(results)

In [71]:
def parse_make(filename):
    with open(filename) as f:
        content = f.readlines()
        
    content = [x.strip() for x in content]
    results = []
    # Set initial data and target to none since the re.match may not find anything
    data = {'Target':None}
    data_list = []
    target = None
    
    for line in content:
        try:
            # If nothing is found, the .group(0) call will cause an error and
            # move to the exception clause
            target = re.match("Scanning dependencies of target [A-z or 0-9 or _]+|[A-z or 0-9 or _]+.so", line,
                              flags=re.IGNORECASE).group(0)[32:]
            # If there is a new target
            if data['Target'] != target:
                # And the current target is not None
                if data['Target']:
                    # Add the current data to the result set
                    results.append(data)
                # No matter what the target is clear the data_list and update the next data point with
                # the new target, empty data_list, and set built to true
                data_list = []
                data = {'Target':target, 'data_list':data_list, 'built':True}
                
        except Exception as e:
            # If there is a dependency we are building
            if target:
                # Check to see if the line has some kind of error or warning
                error_line = re.search("fatal error:|error:|warning:", line, flags=re.IGNORECASE)
                # If there is an error it "probably" failed to build (Hasn't been check with all
                # of the various errors we can get)
                if error_line:
                    data['built'] = False
                data_list.append(line)
            pass
    results.append(data)
    return pd.DataFrame(results)

In [74]:
unittest_logs = parse_logs('unittest.log')
apptest_logs = parse_logs('apptest.log')
make_log = parse_make('make2.log')
make_log

,Target,built,data_list
0,isis3,False,[[ 10%] Building CXX object src/CMakeFiles/isi...


In [75]:
make_log = parse_make('make1.log')
make_log

,Target,built,data_list
0,isis3,True,[[ 10%] Building CXX object src/CMakeFiles/isi...
1,AdaptiveGruen,True,[[ 41%] Building CXX object src/CMakeFiles/Ada...
2,AerialPhotoCamera,True,[[ 41%] Building CXX object src/CMakeFiles/Aer...
3,Albedo,True,[[ 41%] Building CXX object src/CMakeFiles/Alb...
4,AlbedoAtm,True,[[ 41%] Building CXX object src/CMakeFiles/Alb...
5,Anisotropic1,True,[[ 41%] Building CXX object src/CMakeFiles/Ani...
6,Anisotropic2,True,[[ 41%] Building CXX object src/CMakeFiles/Ani...
7,apollo,True,[[ 41%] Building CXX object src/CMakeFiles/apo...
8,ApolloMetricCamera,True,[[ 41%] Building CXX object src/CMakeFiles/Apo...
9,ApolloPanoramicCamera,True,[[ 41%] Building CXX object src/CMakeFiles/Apo...


In [76]:
unittest_logs[~unittest_logs.passed]

,name,passed,test_id,time
280,isis3_unit_test_IException,False,281,0.30
301,isis3_unit_test_SqlQuery,False,302,1.30
302,isis3_unit_test_DatabaseFactory,False,303,0.30
303,isis3_unit_test_Database,False,304,1.30
305,isis3_unit_test_SqlRecord,False,306,1.28
327,isis3_unit_test_MosaicSceneWidget,False,328,1.36


In [270]:
apptest_logs[~apptest_logs.passed]

,name,passed,test_id,time
223,fits_app_test_infoAll,False,554,0.32


In [278]:
print("TOTAL TIME = {}".format(unittest_logs['time'].sum()))
print("AVERAGE TIME = {}".format(unittest_logs['time'].mean()))

TOTAL TIME = 288.92999999999995
AVERAGE TIME = 0.8755454545454544


In [279]:
failed = results[~results.passed]
failed

,name,passed,test_id,time
280,isis3_unit_test_IException,False,281,0.30
301,isis3_unit_test_SqlQuery,False,302,1.30
302,isis3_unit_test_DatabaseFactory,False,303,0.30
303,isis3_unit_test_Database,False,304,1.30
305,isis3_unit_test_SqlRecord,False,306,1.28
327,isis3_unit_test_MosaicSceneWidget,False,328,1.36


In [295]:
msg = MIMEMultipart("TEST ATTACH")
msg['Subject'] = 'EMAIL TEST'
msg['From'] = 'krodriguez@usgs.gov'
msg['To'] = 'krodriguez@usgs.gov'

unittest_logs['type'] = 'unit'
apptest_logs['type'] = 'app'

df = unittest_logs[~unittest_logs.passed].append(apptest_logs[~apptest_logs.passed])
df = df.set_index(['type', 'test_id'])

html = df.to_html().replace("\n", "").replace('<th></th>', '').replace('</tr>    <tr>', '')
attachment = MIMEText(html, 'html')
msg.attach(attachment)

In [296]:
server = smtplib.SMTP('smtp.gmail.com',587) #port 465 or 587
server.ehlo()
server.starttls()
server.ehlo()
server.login('kelvinrodriguez1105@gmail.com','Axel1105')
server.sendmail(msg['From'], [msg['From']], msg.as_string())
server.quit()

(221, b'2.0.0 closing connection b12sm2961152qkc.56 - gsmtp')

In [306]:
df.to_html().replace("\n", "")

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>      <th>name</th>      <th>passed</th>      <th>time</th>    </tr>    <tr>      <th>type</th>      <th>test_id</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th rowspan="6" valign="top">unit</th>      <th>281</th>      <td>isis3_unit_test_IException</td>      <td>False</td>      <td>0.30</td>    </tr>    <tr>      <th>302</th>      <td>isis3_unit_test_SqlQuery</td>      <td>False</td>      <td>1.30</td>    </tr>    <tr>      <th>303</th>      <td>isis3_unit_test_DatabaseFactory</td>      <td>False</td>      <td>0.30</td>    </tr>    <tr>      <th>304</th>      <td>isis3_unit_test_Database</td>      <td>False</td>      <td>1.30</td>    </tr>    <tr>      <th>306</th>      <td>isis3_unit_test_SqlRecord</td>      <td>False</td>      <td>1.28</td>    </tr>    <tr>      <th>328</th>      <td>isis3_unit_test_MosaicSceneWidget</td>      

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>      <th>name</th>      <th>passed</th>      <th>time</th>    </tr>    <tr>      <th>type</th>      <th>test_id</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th rowspan="6" valign="top">unit</th>      <th>281</th>      <td>isis3_unit_test_IException</td>      <td>False</td>      <td>0.30</td>    </tr>    <tr>      <th>302</th>      <td>isis3_unit_test_SqlQuery</td>      <td>False</td>      <td>1.30</td>    </tr>    <tr>      <th>303</th>      <td>isis3_unit_test_DatabaseFactory</td>      <td>False</td>      <td>0.30</td>    </tr>    <tr>      <th>304</th>      <td>isis3_unit_test_Database</td>      <td>False</td>      <td>1.30</td>    </tr>    <tr>      <th>306</th>      <td>isis3_unit_test_SqlRecord</td>      <td>False</td>      <td>1.28</td>    </tr>    <tr>      <th>328</th>      <td>isis3_unit_test_MosaicSceneWidget</td>      <td>False</td>      <td>1.36</td>    </tr>    <tr>      <th>app</th>      <th>554</th>      <td>fits_app_test_infoAll</td>      <td>False</td>      <td>0.32</td>    </tr>  </tbody></table>

In [297]:
html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">                  <th>name</th>      <th>passed</th>      <th>time</th>          <th>type</th>      <th>test_id</th>                      </tr>  </thead>  <tbody>    <tr>      <th rowspan="6" valign="top">unit</th>      <th>281</th>      <td>isis3_unit_test_IException</td>      <td>False</td>      <td>0.30</td>          <th>302</th>      <td>isis3_unit_test_SqlQuery</td>      <td>False</td>      <td>1.30</td>          <th>303</th>      <td>isis3_unit_test_DatabaseFactory</td>      <td>False</td>      <td>0.30</td>          <th>304</th>      <td>isis3_unit_test_Database</td>      <td>False</td>      <td>1.30</td>          <th>306</th>      <td>isis3_unit_test_SqlRecord</td>      <td>False</td>      <td>1.28</td>          <th>328</th>      <td>isis3_unit_test_MosaicSceneWidget</td>      <td>False</td>      <td>1.36</td>          <th>app</th>      <th>554</th>      <td>fits_app_test_infoAll</td>      <td>Fals